In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%autosave 0

In [ ]:
from pathlib import Path
from measurements_table import create_measurements_table
import utils
import pandas
import plotly.express as px

# Read data

In [ ]:
IV_measurements_table_df = create_measurements_table().query('Type=="IV curve"')

measured_data_df = pandas.DataFrame()
for measurement_name in IV_measurements_table_df.index:
    df = pandas.read_feather(utils.path_to_measurements_directory/Path(measurement_name)/Path('IV_curve/measured_data.fd'))
    df['Bias voltage (V)'] *= -1
    df['Bias current (A)'] *= -1
    df['Measurement name'] = measurement_name
    device_name = IV_measurements_table_df.loc[measurement_name, 'Measured device']
    df['Device'] = device_name
    df['Device specs'] = utils.bureaucrat.get_device_specs_string(device_name, humanize=True)
    for param in {'Trench depth','Trenches','Trench process','Pixel border','Contact type','Wafer'}:
        df[param] = utils.bureaucrat.get_devices_specs_dictionary(device_name)[param.lower()]
    measured_data_df = measured_data_df.append(df, ignore_index=True)

In [ ]:
display(measured_data_df)

Calculate mean and std at each `n_voltage`:

In [ ]:
mean_std_df = utils.mean_std(measured_data_df, by=['n_voltage', 'Measurement name','Device','Device specs','Pixel border','Trench depth','Contact type','Trench process','Trenches','Wafer'])
display(mean_std_df)

# Plots

In [ ]:
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = mean_std_df.sort_values(['Device','n_voltage']).reset_index(),
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    error_y = 'Bias current (A) std',
    error_y_mode = 'band',
    symbol = 'Device',
    #line_dash = 'Trench depth',
    hover_name = 'Measurement name',
    log_y = True,
    line_group = 'Measurement name',
)
fig.show()

## Plots for the RD50 Workshop

In [ ]:
df = mean_std_df.sort_values(['Trenches','Device','n_voltage']).reset_index()
df = df.loc[~df['Device'].isin({ # Remove dead devices.
    '1',
    '2',
    '16',
    '21',
    '48',
    '55',
    '56',
    '80',
    '81',
    '86',
    '17', 
    '77',
})]
df = df.loc[~df['Measurement name'].isin({ # Remove some specific measurements.
    '20211024131707_#65_IV_curve',
    '20211023104738_#45_IVCurve', # This is one of the first measurements I did. Later on I re-measured #45 and it was much better, don't know what happened in this particular measurement.
})]
df['Trenches'] = df['Trenches'].astype(str)
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = df,
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    error_y = 'Bias current (A) std',
    error_y_mode = 'band',
    color = 'Trench depth',
    symbol = 'Pixel border',
    line_dash = 'Contact type',
    facet_col = 'Trenches',
    hover_name = 'Measurement name',
    line_group = 'Measurement name',
    grouped_legend = True,
    log_y = True,
)
fig.show()

In [ ]:
fig.write_html(f'iv_curves.html', include_plotlyjs='cdn')

# The death of device #2

In [ ]:
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = mean_std_df.query('Device=="2"').sort_values(['Device','n_voltage']).reset_index(),
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    symbol = 'Device',
    hover_name = 'Measurement name',
    log_y = True,
    line_group = 'Measurement name',
    animation_frame = 'Measurement name'
)
fig.show()